<a href="https://colab.research.google.com/github/mrhamedani/LLM-Agents/blob/main/7_LLAMA2_Moderation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Model: Llama-2-7b-chat-hf / meta-llama/Meta-Llama-3.1-8B-Instruct

Colab Environment: GPU-T4 High-RAM. (If you wwant to test Llama-3.1 you'll need a bigger GPU).

Keys:
* Lagchain
* Llama
* Moderator.


# How To Create a Moderation System Using LangChain & Hugging Face.

We are building a comment moderation system using two language models:

1️ The first model reads user comments and generates a response.

2️⃣ The second model reviews the response. If it detects negativity or inappropriate tone, it modifies the response.

This ensures that negative user comments do not trigger negative replies, keeping the system's responses appropriate.

In [ ]:
#Install de LangChain and openai libraries.
!pip install -q langchain==0.2.11
!pip install -q transformers==4.43.1
!pip install -q accelerate==0.32.1
!pip install -q langchain-community==0.2.10
!pip install -q huggingface_hub  #==0.24.1

In [ ]:
from getpass import getpass
hf_key = getpass("Hugging Face Key: ")

In [ ]:
!huggingface-cli login --token $hf_key

## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.

In [ ]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain.llms import HuggingFacePipeline
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from torch import cuda

In [ ]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device

##Load the Model .

In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
#model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
# begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    token=hf_key)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
    token=hf_key)
model.eval()
print(f"Model loaded on {device}")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id,use_aut_token=hf_key)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    temperature=0.1,
    #do_sample=False,
    top_p=0,
    #trust_remote_code=True,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    device_map='auto')

assistant_llm = HuggingFacePipeline(pipeline=pipe)

## Create the template for the first model called assistant.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment.

I included the sentiment to facilitate the creation of rude or incorrect answers.

In [ ]:
# Instruction how the LLM must respond the comments,
assistant_template = """
[INST]<<SYS>>You are {sentiment} assistant that responds to user comments,
using similar vocabulary than the user.
Stop answering text after answer the first user.<</SYS>>

User comment:{customer_request}[/INST]
assistant_response
"""

In [ ]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

Now we create a First Chain. Just chaining the ***assistant_prompt_template*** and the model. The model will receive the prompt generated with the prompt_template.

In [ ]:
output_parser = StrOutputParser()
assistant_chain = assistant_prompt_template | assistant_llm | output_parser

To execute the chain created it's necessary to call the ***.run*** method of the chain, and pass the variables necessaries.

In our case: ***customer_request*** and ***sentiment***.

In [ ]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #callint the .invoke method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model. To obtain it we are changing the ***sentiment*** variable.

In [ ]:
# This the customer request, or customere comment in the forum moderated by the agent.
# feel free to update it.
customer_request = """Your product is a piece of shit. I want my money back!"""

In [ ]:
# Our assistatnt working in 'nice' mode.
assistant_response=create_dialog(customer_request, "nice")
print(assistant_response)

In [ ]:
#Our assistant running in rude mode.
assistant_response = create_dialog(customer_request, "most rude assistant that exist")
print(assistant_response)

Okay, this answer needs some moderation! Fortunately, we are actively working on it!

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.

In [ ]:
#The moderator prompt template
moderator_template = """
[INST]<<SYS>>You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive an original comment and if it is impolite you must transform into polite.
Try to mantain the meaning when possible.<</SYS>>

Original comment: {comment_to_moderate}/[INST]
"""

# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template
)

In [ ]:
moderator_llm = assistant_llm

In [ ]:
#We build the chain for the moderator.

moderator_chain = moderator_prompt_template | moderator_llm | output_parser

In [ ]:
assistant_response

In [ ]:
# To run our chain we use the .invoke() command
moderator_says = moderator_chain.invoke({"comment_to_moderate": assistant_response})

In [ ]:
print(moderator_says)

This answer is more polite that the one produce by the  **"rude" assistant**.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.



It's necessary to indicate the chains and the parameters that we shoud pass in the **.run** method.

In [ ]:
assistant_moderated_chain = (
    {"comment_to_moderate":assistant_chain}
    |moderator_chain
)

Lets use our Moderating System!

In [ ]:
# We can now run the chain.
from langchain.callbacks.tracers import ConsoleCallbackHandler
assistant_moderated_chain.invoke({"sentiment": "very rude", "customer_request": customer_request},
                                 config={'callbacks':[ConsoleCallbackHandler()]})